In [10]:
!pip install sqlalchemy==1.3.9


  Using cached SQLAlchemy-1.3.9-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.3.9 which is incompatible.


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [8]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [11]:
import pandas as pd

def run_sql(sql_query, database='my_data1.db'):
    """Helper function to execute SQL queries and return results as a Pandas DataFrame."""
    try:
        conn = sqlite3.connect(database)
        df = pd.read_sql_query(sql_query, conn)
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    finally:
        if conn:
            conn.close()
    return df

# SQL query to retrieve unique launch sites
sql_query_task1 = "SELECT DISTINCT Launch_Site FROM SPACEXTBL;"

# Run the query and display results
result_task1 = run_sql(sql_query_task1)

if result_task1 is not None:
    print(result_task1)

    Launch_Site
0   CCAFS LC-40
1   VAFB SLC-4E
2    KSC LC-39A
3  CCAFS SLC-40


In [12]:
sql_query_task2 = "SELECT * FROM SPACEXTBL WHERE Launch_Site LIKE 'CCA%' LIMIT 5;"

result_task2 = run_sql(sql_query_task2)

if result_task2 is not None:
    print(result_task2)

         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

In [13]:
sql_query_task3 = "SELECT SUM(payload_mass__kg_) FROM SPACEXTBL WHERE Customer LIKE '%NASA (CRS)%';"

result_task3 = run_sql(sql_query_task3)

if result_task3 is not None:
    print(result_task3)

   SUM(payload_mass__kg_)
0                   48213


In [14]:
sql_query_task4 = "SELECT AVG(payload_mass__kg_) FROM SPACEXTBL WHERE Booster_Version = 'F9 v1.1';"

result_task4 = run_sql(sql_query_task4)

if result_task4 is not None:
    print(result_task4)

   AVG(payload_mass__kg_)
0                  2928.4


In [15]:
sql_query_task5 = "SELECT MIN(Date) FROM SPACEXTBL WHERE Landing_Outcome = 'Success (ground pad)';"

result_task5 = run_sql(sql_query_task5)

if result_task5 is not None:
    print(result_task5)

    MIN(Date)
0  2015-12-22


In [16]:
sql_query_task6 = "SELECT Booster_Version FROM SPACEXTBL WHERE Landing_Outcome = 'Success (drone ship)' AND payload_mass__kg_ > 4000 AND payload_mass__kg_ < 6000;"

result_task6 = run_sql(sql_query_task6)

if result_task6 is not None:
    print(result_task6)

  Booster_Version
0     F9 FT B1022
1     F9 FT B1026
2  F9 FT  B1021.2
3  F9 FT  B1031.2


In [17]:
sql_query_task7 = """
SELECT
    SUM(CASE WHEN Mission_Outcome LIKE '%Success%' THEN 1 ELSE 0 END) AS Successful,
    SUM(CASE WHEN Mission_Outcome LIKE '%Failure%' THEN 1 ELSE 0 END) AS Failed
FROM SPACEXTBL;
"""

result_task7 = run_sql(sql_query_task7)

if result_task7 is not None:
    print(result_task7)

   Successful  Failed
0         100       1


In [18]:
sql_query_task8 = """
SELECT Booster_Version
FROM SPACEXTBL
WHERE payload_mass__kg_ = (SELECT MAX(payload_mass__kg_) FROM SPACEXTBL);
"""

result_task8 = run_sql(sql_query_task8)

if result_task8 is not None:
    print(result_task8)

   Booster_Version
0    F9 B5 B1048.4
1    F9 B5 B1049.4
2    F9 B5 B1051.3
3    F9 B5 B1056.4
4    F9 B5 B1048.5
5    F9 B5 B1051.4
6    F9 B5 B1049.5
7   F9 B5 B1060.2 
8   F9 B5 B1058.3 
9    F9 B5 B1051.6
10   F9 B5 B1060.3
11  F9 B5 B1049.7 


In [19]:
sql_query_task9 = """
SELECT
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTBL
WHERE
    substr(Date, 1, 4) = '2015' AND
    Landing_Outcome = 'Failure (drone ship)';
"""

result_task9 = run_sql(sql_query_task9)

if result_task9 is not None:
    print(result_task9)

  Month       Landing_Outcome Booster_Version  Launch_Site
0    01  Failure (drone ship)   F9 v1.1 B1012  CCAFS LC-40
1    04  Failure (drone ship)   F9 v1.1 B1015  CCAFS LC-40


In [20]:
sql_query_task10 = """
SELECT
    Landing_Outcome,
    COUNT(*) AS Outcome_Count,
    RANK() OVER (ORDER BY COUNT(*) DESC) AS Outcome_Rank
FROM SPACEXTBL
WHERE
    Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY
    Landing_Outcome
ORDER BY
    Outcome_Count DESC;
"""

result_task10 = run_sql(sql_query_task10)

if result_task10 is not None:
    print(result_task10)

          Landing_Outcome  Outcome_Count  Outcome_Rank
0              No attempt             10             1
1    Success (drone ship)              5             2
2    Failure (drone ship)              5             2
3    Success (ground pad)              3             4
4      Controlled (ocean)              3             4
5    Uncontrolled (ocean)              2             6
6     Failure (parachute)              2             6
7  Precluded (drone ship)              1             8
